## Setup 

In [ ]:
import sys
from scipy.stats import norm
from matplotlib.colors import LogNorm

import importlib
import uproot

import numpy as np
import math
import matplotlib.pyplot as plt

import pandas as pd

import awkward


In [ ]:
# calculations & plotting functions ported to backend script for readability 
import smear_plotter
importlib.reload(smear_plotter)
from smear_plotter import *

In [ ]:
# grab NuMI FHC reduced ntuples with nue preselection: 
# slice ID, containment, & michel electron veto cuts already applied

fold = "nuselection"
tree = "NeutrinoSelectionFilter"

path = '/uboone/data/users/elenag/searchingfornues/NuMISkimmedNtuples/Run1SliceAndSelected/'
FHC_OVRLY = 'prodgenie_numi_uboone_overlay_fhc_mcc9_run1_v28_all_snapshot'

overlay = uproot.open(path+FHC_OVRLY+".root")[fold][tree]

In [ ]:
# create pandas dataframe 

variables = [
    "selected", "nu_pdg", "shr_theta", "true_e_visible", 
    "trk_score_v", "shr_tkfit_dedx_Y", "ccnc", "n_tracks_contained", 
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints",
    "nproton", "nu_e", "n_showers_contained", "nu_purity_from_pfp", 
    "shr_score", 
    "trk_energy", "tksh_distance", "tksh_angle",
    "npi0", "topological_score",
    "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z", "contained_fraction",
    "shrsubclusters0", "shrsubclusters1", "shrsubclusters2",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "npion", "shr_energy_cali", 
    "flash_time", "shrmoliereavg", 
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", "elec_e"
]

overlay_df = overlay.pandas.df(variables, flatten=False)


In [ ]:
# add track PID score 
overlay_df = track_PID_score(overlay, overlay_df)

In [ ]:
overlay_df

## Defining signal & applying a selection

In [ ]:
# define fiducial volume (FV)
true_in_fv_query = "10<=true_nu_vtx_x<=246 and -106<=true_nu_vtx_y<=106 and 10<=true_nu_vtx_z<=1026"
reco_in_fv_query = "10<=reco_nu_vtx_sce_x<=246 and -106<=reco_nu_vtx_sce_y<=106 and 10<=reco_nu_vtx_sce_z<=1026"

# define signal query 
signal = '(nu_pdg==12 and ccnc==0 and nproton>0 and npion==0 and npi0==0)'

In [ ]:
# exclusive electron neutrino selection (CC 1eNp)

# FV cut 
SEL_QUERY = reco_in_fv_query

# signal topology: 1eNp 
SEL_QUERY += ' and n_showers_contained==1'
SEL_QUERY += ' and n_tracks_contained>0'

# numu CC rejection 
SEL_QUERY += ' and shr_score<0.125'
SEL_QUERY += ' and shrmoliereavg < 8'
SEL_QUERY += ' and trkpid<0'

# pi0 rejection 
SEL_QUERY += ' and shr_tkfit_dedx_Y<4'
SEL_QUERY += ' and tksh_distance<5'

In [ ]:
selected = overlay_df.query(SEL_QUERY)

print(str(len(selected.query(signal)))+' signal events were selected')

## Selection efficiency across electron energy 

In [ ]:
# true electron energy 
true_var = 'elec_e'

# reconstructed shower energy 
reco_var = 'shr_energy_cali'

In [ ]:
# detector resolution of selected signal events
det_res = np.array((selected.query(signal)[true_var]-selected.query(signal)[reco_var]))
det_res = det_res/selected.query(signal)[true_var]

# variance 
mu = sum(det_res) / len(det_res) 
sigma = (sum([((x - mu) ** 2) for x in det_res]) / len(det_res))**0.5

print('mu == '+str(round(mu, 3)))
print('sigma == '+str(round(sigma, 3)))

# plot
plot_det_res(det_res, mu, sigma)

# NOTE: sigma is the smallest bin width we should have

In [ ]:
# let's try a constant bin size 

bins = [0.05, 0.25, 0.45, 0.65, 0.85, 1.05, 1.25, 1.45, 1.65, 1.85, 2.05, 2.25, 2.45, 2.65, 2.85, 3.05]

plot_signal_and_eff(selected, overlay_df, signal, bins)

# NOTE: this efficiency is relative to the number of signal events passing our preselection 
# (since we are using the reduced ntuples)

In [ ]:
# the efficiency is quite varied, & the number of signal events in higher bins is low 
# try an adjusted binning based on these observations

bins = [0.05, 0.35, 0.55, 0.75, 0.95, 1.15, 1.65, 3.05]
plot_signal_and_eff(selected, overlay_df, signal, bins)

## Smearing matrix of the electron (shower) energy

In [ ]:
# relationship between true & reco values
plot_smearing(selected, signal, true_var, reco_var, bins)

In [ ]:
# column-normalized 
plot_smearing(selected, signal, true_var, reco_var, bins, norm=True)